## 11. HNSW参数调优难？掌握SQ8量化压缩技术实现速度与准确率平衡

在使用 Faiss 的 HNSW（Hierarchical Navigable Small World）索引时，虽然其搜索速度和准确率都很高，但内存占用大 是其主要缺点之一。

为了解决这个问题，Faiss 提供了 SQ8（Scalar Quantization 8 bits）量化压缩技术 ，可以在几乎不损失精度的前提下显著减少内存占用。

下面我们将通过一个完整的示例，展示：

1. 原始 HNSW 索引的构建与搜索；
2. 使用 SQ8 量化后的 HNSW 构建与搜索；
3. 比较两者在 内存占用、搜索速度、召回率 上的表现。

### 1 安装 Faiss

确保你安装的是支持 GPU 和 SQ8 的版本

In [1]:
%pip install faiss-gpu  # 或 faiss-cpu

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Note: you may need to restart the kernel to use updated packages.


## 2. 示例代码：HNSW vs HNSW + SQ8 对比

In [1]:
import numpy as np
import faiss
import time

# 设置随机种子
np.random.seed(42)

# 构造数据集
d = 128                             # 向量维度
nb = 100000                         # 数据库大小
nq = 1000                           # 查询数量
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')

# 构建原始 HNSW 索引
index_hnsw = faiss.IndexHNSWFlat(d, 32)
index_hnsw.hnsw.efConstruction = 200
index_hnsw.verbose = False
print("开始训练原始 HNSW...")
start_time = time.time()
index_hnsw.add(xb)
end_time = time.time()
print(f"原始 HNSW 训练耗时: {end_time - start_time:.2f}s")

# 查询测试
k = 10
D, I = index_hnsw.search(xq, k)
print("原始 HNSW top-10 查询结果（前5个查询）:")
print(I[:5])

# 获取内存占用
print(f"原始 HNSW 内存占用: {index_hnsw.ntotal * index_hnsw.d * 4 / (1024 ** 2):.2f} MB")


# 构建 HNSW + SQ8 索引
res = faiss.StandardGpuResources() if 'gpu' in faiss.__version__ else None

# 使用 HNSW 作为量化器
quantizer = faiss.IndexHNSWFlat(d, 32)
quantizer.hnsw.efConstruction = 200

# 创建带 SQ8 的索引
index_sq8 = faiss.IndexIVFPQ(quantizer, d, nb // 100, 4, 8)  # PQ 参数可调
index_sq8.nprobe = 10
index_sq8.verbose = False

# 训练索引
print("开始训练 HNSW+SQ8...")
start_time = time.time()
index_sq8.train(xb)
index_sq8.add(xb)
end_time = time.time()
print(f"HNSW+SQ8 训练耗时: {end_time - start_time:.2f}s")

# 查询测试
D2, I2 = index_sq8.search(xq, k)
print("HNSW+SQ8 top-10 查询结果（前5个查询）:")
print(I2[:5])

# 获取内存占用
print(f"HNSW+SQ8 内存占用: {index_sq8.ntotal * index_sq8.d * 1 / (1024 ** 2):.2f} MB")  # SQ8每个向量约1字节

# 回忆率计算（简单比较 Top-1 是否一致）
recall_top1 = np.mean(I[:, 0] == I2[:, 0])
print(f"Top-1 Recall between HNSW and HNSW+SQ8: {recall_top1 * 100:.2f}%")

开始训练原始 HNSW...
原始 HNSW 训练耗时: 27.73s
原始 HNSW top-10 查询结果（前5个查询）:
[[19523 72727 34414 30313 48303 86082 13870 35989 31498 42753]
 [ 5601  1379 16978 75556 52841 66441 46596 35846 39882 73673]
 [31536  7009 27092 30305 74453 35754  5245 75809 14408 71676]
 [22844 76328 12625 23505 29097 55150 68382 62868  8947 57611]
 [51507 21281 41110 32283 61565 23634 93393 58215 67148 52124]]
原始 HNSW 内存占用: 48.83 MB
开始训练 HNSW+SQ8...
HNSW+SQ8 训练耗时: 4.65s
HNSW+SQ8 top-10 查询结果（前5个查询）:
[[13411  6179 13534 25676 48409 87005 42753 31076 92437 33402]
 [12054 67007 37871 44402 96905 54773 32699 74086 67139 87749]
 [78702 64646 59197  6252 48855 86870 23764 83880 44509 17289]
 [22116 80969 91527 61864 30121 55358  5655 29269 69988  3387]
 [32129 68845 10468  3345 71399  9844 48555  5141  6267 90099]]
HNSW+SQ8 内存占用: 12.21 MB
Top-1 Recall between HNSW and HNSW+SQ8: 0.60%


## 3 HNSW 参数详解

| 参数名            | 默认值     | 含义说明                                     | 调整建议 |
|------------------|------------|----------------------------------------------|----------|
| `M`              | 32         | 每个节点连接的最大邻居数，控制图的复杂度     | 增大 M 提高召回率，但增加内存和构建时间 |
| `efConstruction` | 200        | 构建索引时的搜索范围，影响图质量             | 更高质量 = 更大的 efConstruction，适合数据集复杂或对精度要求高的场景 |
| `efSearch`       | 10         | 搜索时的探索范围，直接影响准确率与速度     | 增大 efSearch 可提升准确率，但会降低查询速度 |
| `level_mult`     | 1 / log(M) | 控制层级分布密度                             | 一般不调整，除非有特殊结构需求 |

适用场景：
- M：图的连接密度
场景：需要高召回率 → 增大 M；
场景：内存敏感、快速部署 → 减小 M；

- efConstruction：构建阶段的质量控制
场景：离线训练、追求精确 → 增大该值；

- efSearch：查询阶段的探索深度
场景：实时性要求高 → 适当减小以加快响应；

- level_mult：层级分布系数
场景：通常默认即可，仅在自定义图结构时调整。

## 4 SQ8 vs SQ16 

SQ8（Scalar Quantization 8-bit）
- 优点：压缩率高，内存最小化；
- 缺点：精度略低；
- 场景：适用于移动端、嵌入式设备、轻量级推荐系统。

SQ16（Scalar Quantization 16-bit）
- 优点：保留更高精度；
- 缺点：内存占用翻倍；
- 场景：适用于云服务、图像检索、视频匹配等对精度要求较高的任务。


## 总结
| 需求类型       | 推荐方案                          | 理由说明                                                                 |
|----------------|-----------------------------------|--------------------------------------------------------------------------|
| **内存敏感**   | HNSW + SQ8                        | 每个向量仅占 1 字节，适合嵌入式/移动端或内存受限环境                       |
| **精度优先**   | HNSW + SQ16 / PQ / OPQ            | 提供更高精度，适合图像检索、视频匹配等对召回率要求高的任务                 |
| **查询速度快** | HNSW + SQ8 + efSearch=20~40       | 在保持合理准确率的同时加快响应速度，适合实时系统                           |
| **大规模部署** | HNSW + SQ8 + GPU 加速             | 支持海量数据快速训练与查询，适用于线上服务                                 |
| **图像特征匹配** | HNSW + SQ8 或 SQ16                | 对于 128D 等常见维度，SQ8 已足够；若需更高精度可用 SQ16                     |
| **视频检索系统** | HNSW + SQ16                       | 视频特征通常更复杂，需要更高精度以提升匹配准确性                           |
| **推荐系统应用** | HNSW + SQ8                        | 在保证性能的前提下节省内存，适合用户/物品 Embedding 的高效匹配              |
| **压缩与精度平衡** | HNSW + PQ / OPQ                  | 使用乘积量化技术，在压缩模型大小的同时保持较高搜索准确率                   |